In [ ]:
#writer pouya azizzadh from datrise

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install rasterio
!pip install geopandas matplotlib
!pip install pywavelets
!pip install openpyxl

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import os
import geopandas as gpd
import rasterio
from rasterio.features import geometry_mask
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [ ]:
import pandas as pd
import numpy as np

file_path = "/content/drive/MyDrive/spi(1981- 2024).xlsx"

data = pd.read_excel(file_path)

regions = data['stcode'].unique()

dataframes = []

for region in regions:
    region_data = data[data['stcode'] == region]

    dataframes.append(region_data[["SPI12","SPEI12","tavg-monthy","thorn-monthy", "p-pet-monthy"]])

conditional_dataset = np.mean(dataframes, axis=0)




In [ ]:
data.head()

,stcode,year,month,p-pet,SPEI12,p,SPI12,p-monthy,tavg-monthy,thorn-monthy,p-pet-monthy
0,40706,1981,1,-502.697569,0.224685,273.086626,0.201716,32.921828,-0.197978,0.000000,32.921828
1,40706,1981,2,-510.886024,0.146129,268.229593,0.141658,21.039823,2.676500,3.452688,17.587135
2,40706,1981,3,-506.247172,0.196094,279.223805,0.277859,47.530458,7.369631,19.462662,28.067796
3,40706,1981,4,-417.424279,0.803169,356.383936,1.052638,116.055473,9.642099,31.447074,84.608399
4,40706,1981,5,-343.759258,1.249635,411.037469,1.609581,77.991363,14.775899,67.248563,10.742800


In [ ]:
import os
import numpy as np
import rasterio
import rasterio.warp
import geopandas as gpd
import torch
# بارگذاری تصاویر TIFF

tiff_folder = "/content/drive/MyDrive/1981-2024-IMG"
spi_data = []
for filename in sorted(os.listdir(tiff_folder)):
    if filename.endswith('.tif') or filename.endswith('.tiff'):
        with rasterio.open(os.path.join(tiff_folder, filename)) as src:
            img = src.read(1)
            if img.shape != (200, 200):
                dst_transform = rasterio.transform.from_bounds(*src.bounds, width=200, height=200)
                destination = np.zeros((200, 200), dtype=img.dtype)
                rasterio.warp.reproject(
                    source=img,
                    destination=destination,
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=dst_transform,
                    dst_crs=src.crs,
                    resampling=rasterio.warp.Resampling.bilinear
                )
                img = destination
            spi_data.append(img)

spi_data = np.nan_to_num(np.array(spi_data), nan=np.nanmean(spi_data))

spi_data = torch.tensor(spi_data, dtype=torch.float32).unsqueeze(1).to(device)






In [ ]:
condition_length =7
num_features = conditional_dataset.shape[1]
conditions = []
for i in range(len(spi_data) - condition_length):
    cond = pd.DataFrame(conditional_dataset[i:i + condition_length]).values
    cond = cond.reshape(1, condition_length, num_features)
    conditions.append(cond)

conditions = torch.tensor(np.concatenate(conditions, axis=0), dtype=torch.float32).to(device)





In [ ]:


assert len(spi_data) == len(conditions) + condition_length, "تعداد تصاویر و شرط‌ها باید تطابق داشته باشد"
spi_data = spi_data[condition_length:]  # برش تصاویر برای تطبیق







In [ ]:




from sklearn.model_selection import train_test_split

spi_data_np = spi_data.cpu().numpy()
conditions_np = conditions.cpu().numpy()

# تقسیم داده‌ها به train و test
spi_train, spi_test, cond_train, cond_test = train_test_split(
    spi_data_np, conditions_np, test_size=0.2, random_state=42
)

spi_train = torch.tensor(spi_train, dtype=torch.float32).to(device)
spi_test = torch.tensor(spi_test, dtype=torch.float32).to(device)
cond_train = torch.tensor(cond_train, dtype=torch.float32).to(device)
cond_test = torch.tensor(cond_test, dtype=torch.float32).to(device)

train_dataset = TensorDataset(spi_train, cond_train)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

from torch.utils.data import SubsetRandomSampler, DataLoader
dataset_size = len(train_dataset)
indices = list(range(dataset_size))
split = int(0.2 * dataset_size)  # 20% برای اعتبارسنجی
train_indices, val_indices = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)
train_dataloader = DataLoader(train_dataset, batch_size=8, sampler=train_sampler)
val_dataloader = DataLoader(train_dataset, batch_size=8, sampler=val_sampler)


test_dataset = TensorDataset(spi_test, cond_test)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)





In [ ]:
import torch
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, latent_dim=100, condition_dim=4, image_shape=(1, 200, 200)):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        self.condition_dim = condition_dim
        self.image_shape = image_shape

        self.fc1 = nn.Linear(latent_dim + condition_length * num_features, 512) s
        self.fc2 = nn.Linear(512, 128 * 25 * 25)

        self.model = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            nn.ConvTranspose2d(64, 32, 4, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.2),
            nn.ConvTranspose2d(32, 1, 4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, z, condition):
        condition = condition.view(condition.size(0), -1)
        input = torch.cat([z, condition], dim=1)
        x = self.fc1(input)
        x = nn.LeakyReLU(0.2)(x)
        x = self.fc2(x)
        x = x.view(-1, 128, 25, 25)
        img = self.model(x)
        return img

class Discriminator(nn.Module):
    def __init__(self, image_shape=(1, 200, 200), condition_dim=4):
        super(Discriminator, self).__init__()
        self.condition_dim = condition_dim

        self.conv = nn.Sequential(
            nn.Conv2d(1 + 1, 64, 4, stride=2, padding=1),  # +1 for condition channel
            nn.LeakyReLU(0.1),
            nn.Dropout(0.3),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Conv2d(128, 256, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )

        dummy_input = torch.randn(1, 2, image_shape[1], image_shape[2])
        dummy_output = self.conv(dummy_input)
        linear_input_size = dummy_output.view(1, -1).shape[1]

        self.fc = nn.Sequential(
            nn.Linear(linear_input_size, 1),
            nn.Dropout(0.4),
            nn.Sigmoid()
        )

    def forward(self, img, condition):
        cond_channel = condition[:, :, 0].unsqueeze(1).unsqueeze(2)

        cond_channel = cond_channel.repeat(1, 1, img.shape[2], img.shape[3])
        cond_channel = torch.nn.functional.interpolate(cond_channel, size=(img.shape[2], img.shape[3]), mode='bilinear', align_corners=False)

        input_data = torch.cat([img, cond_channel], dim=1)

        x = self.conv(input_data)

        x = x.view(x.size(0), -1)

        validity = self.fc(x)

        return validity



In [ ]:
import torch
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from sklearn.metrics import r2_score, mean_squared_error
import cv2
import numpy.ma as ma

mask_path = '/content/drive/MyDrive/iran.jpg'
mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
mask = cv2.resize(mask, (200, 200), interpolation=cv2.INTER_NEAREST)
mask = mask / 255.0
mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
mask = cv2.resize(mask, (200, 200), interpolation=cv2.INTER_NEAREST)
mask = mask / 255.0
mask_resized = cv2.resize(mask, (200, 200), interpolation=cv2.INTER_NEAREST)

In [ ]:

import torch
import torch.nn as nn
from sklearn.metrics import r2_score

# تعریف مدل‌ها
latent_dim = 120
condition_dim = 7
generator = Generator(latent_dim, condition_dim).to(device)
discriminator = Discriminator(condition_dim=condition_dim).to(device)
num_epochs=50
optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.002, betas=(0.5, 0.999), weight_decay=1e-4)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0001, betas=(0.5, 0.999), weight_decay=1e-4)

adversarial_loss = nn.BCELoss()

best_val_loss = 0
counter = 0

optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.002, betas=(0.5, 0.999), weight_decay=1e-4)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0001, betas=(0.5, 0.999), weight_decay=1e-4)
scheduler_G = torch.optim.lr_scheduler.StepLR(optimizer_G, step_size=15, gamma=0.2)
scheduler_D = torch.optim.lr_scheduler.StepLR(optimizer_D, step_size=15, gamma=0.2)

lambda_mse = 15.0
patience = 500

for epoch in range(num_epochs):
    for i, (real_images, cond) in enumerate(train_dataloader):
        batch_size = real_images.size(0)
        real_images = real_images.to(device)
        cond = cond.to(device)

        optimizer_D.zero_grad()
        real_validity = discriminator(real_images, cond)
        real_loss = adversarial_loss(real_validity, torch.full((batch_size, 1), 0.8).to(device))
        z = torch.randn(batch_size, latent_dim).to(device)
        fake_images = generator(z, cond)

        fake_validity = discriminator(fake_images.detach(), cond)
        fake_loss = adversarial_loss(fake_validity, torch.full((batch_size, 1), 0.8).to(device))
        d_loss = (real_loss + fake_loss)
        d_loss.backward()
        optimizer_D.step()

        optimizer_G.zero_grad()
        fake_validity = discriminator(fake_images, cond)
        adv_loss = adversarial_loss(fake_validity, torch.ones(batch_size, 1).to(device))
        mse_loss = torch.mean((real_images - fake_images) ** 2)
        g_loss = adv_loss + lambda_mse * mse_loss
        g_loss.backward()
        optimizer_G.step()

    scheduler_G.step()
    scheduler_D.step()

    val_g_loss = 0
    val_r2 = []
    with torch.no_grad():
        for val_images, val_cond in val_dataloader:
            val_images = val_images.to(device)
            val_cond = val_cond.to(device)
            z_val = torch.randn(val_images.size(0), latent_dim).to(device)
            fake_val_images = generator(z_val, val_cond)
            val_g_loss += torch.mean((val_images - fake_val_images) ** 2).item()
            val_r2.append(r2_score(val_images.cpu().numpy().flatten(), fake_val_images.cpu().numpy().flatten()))
    val_g_loss /= len(val_dataloader)
    avg_val_r2 = np.mean(val_r2)

    print(f"Epoch [{epoch}/{num_epochs}] | D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f} | Val Loss: {val_g_loss:.4f} | Val R²: {avg_val_r2:.4f}")

    if avg_val_r2 >= 0.39:
        print(f"R² به {avg_val_r2:.4f} رسید!")
        break

    if val_g_loss < best_val_loss:
        best_val_loss = val_g_loss
        counter = 0
        torch.save(generator.state_dict(), '/content/drive/MyDrive/best_generator.pth')
    else:
        counter += 1
        if counter >= patience:
            print(f"Early Stopping در epoch {epoch} اعمال شد.")
            break



Epoch [0/50] | D Loss: 1.1554 | G Loss: 20.2704 | Val Loss: 1.8698 | Val R²: 0.3214
Epoch [1/50] | D Loss: 1.2458 | G Loss: 23.5158 | Val Loss: 1.7409 | Val R²: 0.3291
Epoch [2/50] | D Loss: 1.2423 | G Loss: 34.5427 | Val Loss: 1.7364 | Val R²: 0.3335
Epoch [3/50] | D Loss: 1.1498 | G Loss: 36.0659 | Val Loss: 1.7555 | Val R²: 0.3273
Epoch [4/50] | D Loss: 1.1954 | G Loss: 14.6820 | Val Loss: 1.7571 | Val R²: 0.3287
Epoch [5/50] | D Loss: 1.0976 | G Loss: 13.0769 | Val Loss: 1.8180 | Val R²: 0.3309
Epoch [6/50] | D Loss: 1.2419 | G Loss: 14.8003 | Val Loss: 1.7426 | Val R²: 0.3384
Epoch [7/50] | D Loss: 1.0986 | G Loss: 23.2786 | Val Loss: 1.7805 | Val R²: 0.3380
Epoch [8/50] | D Loss: 1.0983 | G Loss: 17.3913 | Val Loss: 1.6760 | Val R²: 0.3413
Epoch [9/50] | D Loss: 1.0979 | G Loss: 12.5595 | Val Loss: 1.7477 | Val R²: 0.3271
Epoch [10/50] | D Loss: 1.0973 | G Loss: 30.9145 | Val Loss: 1.8200 | Val R²: 0.3301
Epoch [11/50] | D Loss: 1.0983 | G Loss: 20.8362 | Val Loss: 1.6873 | Val R

In [ ]:
import pywt

def wavelet_denoise(img, wavelet='db1', level=2, threshold=0.1):
    """
    کاهش نویز تصویر با استفاده از Wavelet Transform
    img: تصویر ورودی (numpy array)
    wavelet: نوع موجک (مثلاً 'db1' برای Daubechies 1)
    level: سطح تجزیه
    threshold: آستانه برای حذف نویز
    """
    coeffs = pywt.wavedec2(img, wavelet, level=level)
    # Modify thresholding to keep the 3-tuple structure
    coeffs = tuple([pywt.threshold(c, threshold, mode='soft') if isinstance(c, np.ndarray) else
                   [pywt.threshold(sub_c, threshold, mode='soft') for sub_c in c]
                   for c in coeffs])

    denoised_img = pywt.waverec2(coeffs, wavelet)
    return denoised_img

def denoise_generated_images(generated_images):
    generated_images = generated_images.detach().cpu().numpy().squeeze()  # Detach before converting to NumPy
    denoised_images = np.zeros_like(generated_images)
    for i in range(generated_images.shape[0]):
        denoised_images[i] = wavelet_denoise(generated_images[i])
    return torch.tensor(denoised_images, dtype=torch.float32).unsqueeze(1).to(device) # Convert back to tensor





In [ ]:
import torch
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from sklearn.metrics import r2_score, mean_squared_error
import numpy.ma as ma
import cv2

def evaluate_model(generator, test_dataloader, mask_path='/content/drive/MyDrive/iran.jpg'):
    generator.eval()
    ssim_scores = []
    psnr_scores = []
    r2_scores = []
    rmse_scores = []
    real_images = []
    pred_images = []

    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    mask_resized = cv2.resize(mask, (200, 200), interpolation=cv2.INTER_NEAREST) / 255.0

    with torch.no_grad():
        for real_imgs, conds in test_dataloader:
            z = torch.randn(real_imgs.size(0), latent_dim).to(device)
            gen_imgs = generator(z, conds)
            gen_imgs_denoised = denoise_generated_images(gen_imgs)

            real_imgs_np = real_imgs.cpu().numpy().squeeze()
            gen_imgs_np = gen_imgs_denoised.cpu().numpy().squeeze()

            real_imgs_np = np.clip(real_imgs_np, 0, 1)
            gen_imgs_np = np.clip(gen_imgs_np, 0, 1)

            for i in range(real_imgs_np.shape[0]):
                real_masked = ma.masked_where(mask_resized == 0, real_imgs_np[i])
                gen_masked = ma.masked_where(mask_resized == 0, gen_imgs_np[i])

                ssim_score = ssim(real_masked.filled(0), gen_masked.filled(0), data_range=1)
                ssim_scores.append(ssim_score)

                psnr_score = psnr(real_masked.filled(0), gen_masked.filled(0), data_range=1)
                psnr_scores.append(psnr_score)

                real_flat = real_masked.filled(0).flatten()
                gen_flat = gen_masked.filled(0).flatten()
                r2_scores.append(r2_score(real_flat, gen_flat))
                rmse_scores.append(np.sqrt(mean_squared_error(real_flat, gen_flat)))

                real_images.append(real_masked.filled(0))
                pred_images.append(gen_masked.filled(0))

    print(f"Average SSIM: {np.mean(ssim_scores):.4f}")
    print(f"Average PSNR: {np.mean(psnr_scores):.2f} dB")
    print(f"Average R²: {np.mean(r2_scores):.4f}")
    print(f"Average RMSE: {np.mean(rmse_scores):.4f}")

    return real_images, pred_images


conds torch.Size([8, 7, 5])
z torch.Size([8, 120])
conds torch.Size([8, 7, 5])
z torch.Size([8, 120])
conds torch.Size([8, 7, 5])
z torch.Size([8, 120])
conds torch.Size([8, 7, 5])
z torch.Size([8, 120])
conds torch.Size([8, 7, 5])
z torch.Size([8, 120])
conds torch.Size([8, 7, 5])
z torch.Size([8, 120])
conds torch.Size([8, 7, 5])
z torch.Size([8, 120])
conds torch.Size([8, 7, 5])
z torch.Size([8, 120])
conds torch.Size([8, 7, 5])
z torch.Size([8, 120])
conds torch.Size([8, 7, 5])
z torch.Size([8, 120])
conds torch.Size([8, 7, 5])
z torch.Size([8, 120])
conds torch.Size([8, 7, 5])
z torch.Size([8, 120])
conds torch.Size([8, 7, 5])
z torch.Size([8, 120])
Average SSIM: 0.8533052002283242
Average PSNR: 4.527991824240273


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, Normalize
import numpy.ma as ma
import cv2
import os

colors = ['brown', 'red', 'orange', 'yellow', 'green', 'blue']
cmap = ListedColormap(colors)

def save_colored_images(real_images, pred_images, mask, output_dir='/content/drive/MyDrive/test_spi_image'):
    """
    تابع برای ذخیره تصاویر واقعی و تولیدی با رنگ‌آمیزی خاص و اعمال ماسک.

    پارامترها:
    - real_images: لیست یا آرایه‌ای از تصاویر واقعی
    - pred_images: لیست یا آرایه‌ای از تصاویر تولیدی
    - mask: آرایه ماسک با مقادیر 0 و 1
    - output_dir: مسیر پوشه‌ای که تصاویر در آن ذخیره می‌شوند
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    mask_resized = cv2.resize(mask, (200, 200), interpolation=cv2.INTER_NEAREST)

    def process_and_save(img, idx, prefix):
        img = img.reshape(200, 200)
        img_min, img_max = img.min(), img.max()

        masked_img = ma.masked_where(mask_resized == 0, img)

        plt.figure(figsize=(5, 5))
        plt.imshow(masked_img, cmap=cmap, norm=Normalize(vmin=img_min, vmax=img_max))
        plt.axis("off")
        plt.colorbar()
        plt.savefig(os.path.join(output_dir, f'{prefix}_colored_{idx}.png'), bbox_inches='tight')
        plt.close()

    for idx, img in enumerate(real_images):
        process_and_save(img, idx, 'real')

    for idx, img in enumerate(pred_images):
        process_and_save(img, idx, 'pred')



In [ ]:
save_colored_images(real_images, pred_images,mask)

In [ ]:
import pandas as pd
import numpy as np
import torch
import os
import rasterio
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, Normalize
import numpy.ma as ma
import cv2
import os
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import matplotlib.cm as cm
import rasterio
from numpy import ma
# تنظیم دستگاه
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ۱. آماده‌سازی داده‌ها
future_file_path = "/content/drive/MyDrive/120_months.xlsx"

# historical_data = pd.read_excel(historical_file_path)
future_data = pd.read_excel(future_file_path)

# تعریف نام ستون‌ها
columns = ["SPI12", "SPEI12", "tavg-monthy", "thorn-monthy", "p-pet-monthy"]

# تبدیل numpy array به pandas DataFrame
historical_data = pd.DataFrame(conditional_dataset, columns=columns)

historical_data = historical_data[["SPI12", "SPEI12", "tavg-monthy", "thorn-monthy", "p-pet-monthy"]]
future_data = future_data[["SPI12", "SPEI12", "tavg-monthy", "thorn-monthy", "p-pet-monthy"]]
spi_values=future_data["SPI12"].values
full_data = pd.concat([historical_data, future_data], ignore_index=True)

condition_length = 7
num_features = 5
conditions_future = []
for t in range(1, 121):
    start_idx = len(historical_data) - (condition_length - (t - 1))
    end_idx = start_idx + condition_length
    cond = full_data.iloc[start_idx:end_idx].values
    conditions_future.append(cond)

conditions_future = np.array(conditions_future)
conditions_future = torch.tensor(conditions_future, dtype=torch.float32).to(device)

# ۲. تعریف و بارگذاری مدل Generator
latent_dim = 120
condition_dim = 7
# generator = Generator(latent_dim, condition_dim).to(device)  # فرض می‌کنیم Generator تعریف شده است
# generator.load_state_dict(torch.load('/content/drive/MyDrive/best_generator.pth'))
generator.eval()

# ۳. تولید تصاویر
z_future = torch.randn(120, latent_dim).to(device)
with torch.no_grad():
    generated_images_future = generator(z_future, conditions_future)

# ۴. کاهش نویز (اختیاری)
generated_images_future = denoise_generated_images(generated_images_future)  # فرض می‌کنیم تابع تعریف شده است

# ۵. ذخیره تصاویر
generated_images_np = generated_images_future.cpu().numpy().squeeze()
output_folder = "/content/drive/MyDrive/future_images"
os.makedirs(output_folder, exist_ok=True)

for i in range(120):
    img = generated_images_np[i]
    with rasterio.open(
        os.path.join(output_folder, f"future_month_{i+1}.tiff"),
        'w',
        driver='GTiff',
        height=200,
        width=200,
        count=1,
        dtype=img.dtype,
        crs='+proj=latlong',
        transform=rasterio.transform.from_bounds(44, 24, 64, 40, 200, 200)
    ) as dst:
        dst.write(img, 1)

print("تصاویر ۱۲۰ ماه آینده با موفقیت تولید و ذخیره شدند!")

# ۶. نمایش تصاویر
# تعریف رنگ‌ها برای ListedColormap
colors = ['brown', 'red', 'orange', 'yellow', 'green', 'blue']
cmap = ListedColormap(colors)

# مسیر ماسک
mask_path = "/content/drive/MyDrive/iran.jpg"  # مسیر ماسک را با مسیر واقعی جایگزین کنید

# بارگذاری و تغییر اندازه ماسک
mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
mask_resized = cv2.resize(mask, (200, 200), interpolation=cv2.INTER_NEAREST)
generated_images_np.shape

def display_future_images(output_folder, mask_resized):
    start_year = 2024
    start_month = 9

    # لیست تمام فایل‌های TIFF در پوشه خروجی
    tiff_files = [f for f in os.listdir(output_folder) if f.endswith('.tiff')]

    # اطمینان از مرتب‌سازی صحیح فایل‌ها
    tiff_files.sort(key=lambda x: int(x.split('_')[2].split('.')[0]))

    # حلقه بر روی فایل‌های ذخیره‌شده
    for i, filename in enumerate(tiff_files):
        # خواندن تصویر از فایل TIFF
        with rasterio.open(os.path.join(output_folder, filename)) as src:
            img = src.read(1)
        img_normalized = (img - img_min) / (img_max - img_min) * (10 - (-5)) + (-5)

        # اعمال ماسک
        masked_img = ma.masked_where(mask_resized == 0, img)

        # محاسبه تاریخ
        total_month = (start_month - 1) + i
        year = start_year + total_month // 12
        month = (total_month % 12) + 1
        date_str = f"{year}_{month:02d}"

        # نمایش و ذخیره تصویر
        fig, ax = plt.subplots(figsize=(5, 5))
        im = ax.imshow(masked_img, cmap=cmap)
        ax.set_title(f"{date_str}_spi_{spi_values[i]}")
        cbar = fig.colorbar(im, ax=ax)
        cbar.set_label("SPI Values")

        # ایجاد پوشه خروجی برای PNGها (اگر وجود ندارد)
        png_output_folder = "/content/drive/MyDrive/2024_10_to_234_9_spi_image"
        os.makedirs(png_output_folder, exist_ok=True)

        # ذخیره تصویر به صورت PNG
        output_filename = os.path.join(png_output_folder, f"{date_str}_spi_image.png")
        plt.savefig(output_filename, bbox_inches='tight', dpi=150)
        plt.close(fig)



تصاویر ۱۲۰ ماه آینده با موفقیت تولید و ذخیره شدند!


In [ ]:
display_future_images(output_folder, mask_resized)